In [3]:
import pandas as pd
import os
from tkinter import filedialog
import tkinter as tk
import re
from openpyxl.utils import get_column_letter
import openpyxl  

def convert_to_minutes(time_value, unit='min'):
    """Convert time values to minutes regardless of input unit"""
    try:
        if unit.lower() in ['sec', 's']:
            return float(time_value) / 60
        return float(time_value)
    except ValueError:
        return None

def extract_metadata_from_filename(filename):
    """Extract metadata specifically from the filename pattern seen in the logs"""
    metadata = {}
    
    # Extract cycles
    cycles_match = re.search(r'(\d+)\s*cycles', filename)
    if cycles_match:
        metadata['cycles'] = cycles_match.group(1)
    
    # Extract replicate number
    replicate_match = re.search(r'replicate_(\d+)', filename)
    if replicate_match:
        metadata['replicate'] = replicate_match.group(1)
    
    # Extract gelatin concentration (if present)
    concentration_match = re.search(r'(\d+)%\s*gelatin', filename)
    if concentration_match:
        metadata['gelatin_conc'] = concentration_match.group(1)
    
    return metadata

def get_particle_type_from_file_path(file_path):
    """Extract particle type from the directory path"""
    path_parts = file_path.split(os.sep)
    
    # Look for particle type: should be after gelatin concentration and particle concentration
    for i, part in enumerate(path_parts):
        if 'gelatin' in part.lower() and '%' in part:
            # Check next few directories after gelatin
            for j in range(i+1, min(i+4, len(path_parts))):
                potential = path_parts[j].strip()
                # Skip concentration folders
                if re.match(r'^\d+\s*ug-ml', potential, re.IGNORECASE):
                    continue
                # Skip common non-particle folders
                if potential.lower() in ['stitched_image', 'metrics', 'merged_analyses']:
                    continue
                if 'replicate' in potential.lower():
                    continue
                if re.match(r'\d+-\d+;\s*\d+\s*cycles', potential):  # Pattern like "1-2; 28 cycles"
                    continue
                # This looks like a particle type
                return potential
    
    return "Unknown"

def get_particle_prefix(particle_type):
    """Get single letter prefix for particle type"""
    particle_upper = particle_type.upper()
    if 'DBPC' in particle_upper and 'HMSN' in particle_upper:
        return 'D'
    elif particle_upper == 'MSN':
        return 'M'
    elif 'HMSN' in particle_upper:
        return 'H'
    else:
        return 'U'  # Unknown

def extract_metadata(file_path):
    """Extract metadata from the file path including both gelatin and particle concentrations"""
    filename = os.path.basename(file_path)
    path_parts = file_path.split(os.sep)
    
    # First try to extract from filename directly
    metadata = extract_metadata_from_filename(filename)
    
    # Extract particle type
    metadata['particle_type'] = get_particle_type_from_file_path(file_path)
    
    # Extract cycles from path or filename
    if 'cycles' not in metadata:
        for part in path_parts + [filename]:
            cycles_match = re.search(r'(\d+)\s*cycles', part.lower())
            if cycles_match:
                metadata['cycles'] = cycles_match.group(1)
                break
    
    # Extract replicate if not already found
    if 'replicate' not in metadata:
        for part in path_parts:
            if 'replicate' in part.lower():
                try:
                    replicate = part.split('replicate_')[1].split('_')[0]
                    metadata['replicate'] = replicate
                except IndexError:
                    replicate = ''.join(filter(str.isdigit, part))
                    if replicate:
                        metadata['replicate'] = replicate
                break
        
        if 'replicate' not in metadata:
            rep_match = re.search(r'rep_(\d+)', filename)
            if rep_match:
                metadata['replicate'] = rep_match.group(1)
    
    # Extract GELATIN concentration from path (look for "X% gelatin")
    if 'gelatin_conc' not in metadata:
        for part in path_parts:
            gelatin_match = re.search(r'(\d+(?:\.\d+)?)\s*%\s*gelatin', part.lower())
            if gelatin_match:
                metadata['gelatin_conc'] = gelatin_match.group(1)
                break
    
    # Extract PARTICLE concentration (ug-mL format)
    for part in path_parts:
        particle_match = re.search(r'(\d+(?:\.\d+)?)\s*ug-ml', part.lower())
        if particle_match:
            metadata['particle_conc'] = particle_match.group(1)
            break
    
    # Extract treatment time
    for part in path_parts:
        if 'min' in part.lower():
            try:
                time_str = part.lower().split('min')[0].strip()
                time_val = convert_to_minutes(time_str, 'min')
                if time_val is not None:
                    metadata['time'] = str(time_val)
                break
            except ValueError:
                continue
        elif 's' in part.lower() and part.lower().replace('s', '').strip().isdigit():
            try:
                time_str = part.lower().replace('s', '').strip()
                time_val = convert_to_minutes(time_str, 's')
                if time_val is not None:
                    metadata['time'] = str(time_val)
                break
            except ValueError:
                continue
    
    return metadata

def create_sheet_name(metadata):
    """Create a sheet name based on available metadata - MAX 31 CHARS"""
    parts = []
    
    # Particle type prefix (1 char: D, M, H, U)
    if 'particle_type' in metadata:
        prefix = get_particle_prefix(metadata['particle_type'])
        parts.append(prefix)
    
    # Gelatin concentration (G# - short)
    if 'gelatin_conc' in metadata:
        parts.append(f"G{metadata['gelatin_conc']}")
    
    # Particle concentration (P# - short)
    if 'particle_conc' in metadata:
        parts.append(f"P{metadata['particle_conc']}")
    
    # Cycles (e.g., 28C)
    if 'cycles' in metadata:
        parts.append(f"{metadata['cycles']}C")
    
    # Treatment time (optional - usually not present)
    if 'time' in metadata:
        parts.append(f"{metadata['time']}m")
    
    # Replicate (R#)
    if 'replicate' in metadata:
        parts.append(f"R{metadata['replicate']}")
    
    sheet_name = '_'.join(parts) if parts else 'Unknown'
    
    # Ensure under 31 character limit
    if len(sheet_name) > 31:
        sheet_name = sheet_name[:31]
        print(f"Warning: Sheet name truncated to 31 chars: {sheet_name}")
    
    return sheet_name

def create_grouping_key(metadata):
    """Create a key for grouping files (by gelatin and particle concentration)"""
    parts = []
    
    if 'gelatin_conc' in metadata:
        parts.append(f"{metadata['gelatin_conc']}pct_gelatin")
    
    if 'particle_conc' in metadata:
        parts.append(f"{metadata['particle_conc']}ugmL")
    
    return '_'.join(parts) if parts else 'ungrouped'

def find_excel_files(directory):
    excel_files = []
    unique_files = {}
    
    for root, dirs, files in os.walk(directory):
        # Filter out directories to skip
        dirs_to_remove = []
        for d in dirs:
            if 'blank' in d.lower() or 'merged_analyses' in d.lower():
                dirs_to_remove.append(d)
        
        for d in dirs_to_remove:
            if d in dirs:
                dirs.remove(d)
        
        for file in files:
            if ('metrics' in root or 'metrics' in file.lower()) and \
               file.endswith('.xlsx') and \
               not file.startswith('~') and \
               'blank_statistics' not in file and \
               'blank' not in root.lower():
                
                full_path = os.path.join(root, file)
                metadata = extract_metadata(full_path)
                
                # Only process if we have required metadata
                if metadata.get('cycles') is not None and metadata.get('replicate') is not None:
                    file_key = full_path
                    
                    if file_key not in unique_files:
                        unique_files[file_key] = (full_path, metadata)
                    else:
                        print(f"Skipping duplicate file: {os.path.basename(full_path)}")
                else:
                    print(f"Warning: Could not extract required metadata from {file}, skipping")
                    print(f"  Path: {full_path}")
                    print(f"  Extracted metadata: {metadata}")
    
    for file_key, (file_path, metadata) in unique_files.items():
        excel_files.append((file_path, metadata))
    
    # Sort files by particle type, gelatin conc, particle conc, cycles, and replicate
    excel_files.sort(key=lambda x: (
        x[1].get('particle_type', 'Unknown'),
        float(x[1].get('gelatin_conc', '0')),
        float(x[1].get('particle_conc', '0')),
        int(x[1]['cycles']),
        int(x[1]['replicate'])
    ))
    
    return excel_files

def autofit_columns(worksheet):
    """Auto-fit columns in a worksheet"""
    for column in worksheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        
        for cell in column:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        
        adjusted_width = min(max_length + 2, 50)
        if adjusted_width > 0:
            worksheet.column_dimensions[column_letter].width = adjusted_width

def merge_excel_files(merge_metrics=True, group_by_concentration=True):
    """
    Main function to merge Excel files
    
    Parameters:
    -----------
    merge_metrics : bool, optional
        Whether to merge Metrics sheets (default True)
    group_by_concentration : bool, optional
        Whether to create separate merged files for each concentration combination (default True)
    """
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    
    selected_dir = filedialog.askdirectory(title="Select Directory")
    root.attributes('-topmost', False)
    
    if not selected_dir:
        print("No directory selected")
        return

    excel_files = find_excel_files(selected_dir)
    if not excel_files:
        print("No Excel files found")
        return
    
    print(f"\nFound {len(excel_files)} files")
    
    if not merge_metrics:
        print("*** Metrics sheets will be SKIPPED ***")
    
    # Print summary of found files grouped by concentrations
    conc_groups = {}
    for _, metadata in excel_files:
        key = create_grouping_key(metadata)
        if key not in conc_groups:
            conc_groups[key] = []
        conc_groups[key].append(metadata)
    
    print("\nFiles by concentration group:")
    for group_key, files in conc_groups.items():
        print(f"  {group_key}: {len(files)} files")
        # Show particle type and cycles breakdown
        particle_counts = {}
        for meta in files:
            particle = meta.get('particle_type', 'Unknown')
            cycles = meta.get('cycles', 'Unknown')
            key = f"{particle} - {cycles} cycles"
            particle_counts[key] = particle_counts.get(key, 0) + 1
        for key, count in sorted(particle_counts.items()):
            print(f"    {key}: {count} replicates")
    
    # Create the merged_analyses folder
    save_dir = os.path.join(selected_dir, 'merged_analyses')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Get all possible sheet names
    all_sheets = set()
    for file_path, _ in excel_files:
        try:
            file_sheets = pd.ExcelFile(file_path).sheet_names
            all_sheets.update(file_sheets)
        except Exception as e:
            print(f"Error reading sheets from {os.path.basename(file_path)}: {e}")
    
    if not merge_metrics:
        all_sheets = {sheet for sheet in all_sheets if sheet.lower() != 'metrics'}
    
    sheet_names = list(all_sheets)
    print(f"\nFound sheets to process: {sheet_names}")

    # Process each sheet type
    for sheet_type in sheet_names:
        if not merge_metrics and sheet_type.lower() == 'metrics':
            continue
        
        sheet_dir = os.path.join(save_dir, sheet_type.replace(" ", "_"))
        if not os.path.exists(sheet_dir):
            os.makedirs(sheet_dir)
        
        print(f"\n{'='*60}")
        print(f"Processing sheet type: {sheet_type}")
        print(f"{'='*60}")
        
        if group_by_concentration:
            # Create separate files for each concentration combination
            for group_key in conc_groups.keys():
                group_files = [(fp, meta) for fp, meta in excel_files 
                              if create_grouping_key(meta) == group_key]
                
                if not group_files:
                    continue
                
                output_file = os.path.join(sheet_dir, 
                                          f'Merged_{sheet_type.replace(" ", "_")}_{group_key}.xlsx')
                
                print(f"\nCreating file for {group_key}...")
                
                with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                    sheets_written = 0
                    
                    for file_path, metadata in group_files:
                        try:
                            sheet_name = create_sheet_name(metadata)
                            
                            try:
                                df = pd.read_excel(file_path, sheet_name=sheet_type)
                                df.to_excel(writer, sheet_name=sheet_name, index=False)
                                sheets_written += 1
                                print(f"  Added sheet: {sheet_name} from {os.path.basename(file_path)}")
                            except ValueError:
                                continue
                        except Exception as e:
                            print(f"  Error processing {os.path.basename(file_path)}: {e}")
                    
                    if sheets_written > 0 and hasattr(writer, 'book'):
                        workbook = writer.book
                        for worksheet in workbook.worksheets:
                            autofit_columns(worksheet)
                
                if sheets_written > 0:
                    print(f"✓ Created: {os.path.basename(output_file)} ({sheets_written} sheets)")
                else:
                    try:
                        os.remove(output_file)
                    except:
                        pass
        else:
            # Create one file with all data
            all_output_file = os.path.join(sheet_dir, f'Merged_{sheet_type.replace(" ", "_")}.xlsx')
            
            with pd.ExcelWriter(all_output_file, engine='openpyxl') as writer:
                sheets_written = 0
                
                for file_path, metadata in excel_files:
                    try:
                        sheet_name = create_sheet_name(metadata)
                        
                        try:
                            df = pd.read_excel(file_path, sheet_name=sheet_type)
                            df.to_excel(writer, sheet_name=sheet_name, index=False)
                            sheets_written += 1
                        except ValueError:
                            continue
                    except Exception as e:
                        print(f"Error processing {os.path.basename(file_path)}: {e}")
                
                if sheets_written > 0 and hasattr(writer, 'book'):
                    workbook = writer.book
                    for worksheet in workbook.worksheets:
                        autofit_columns(worksheet)
            
            if sheets_written > 0:
                print(f"Created merged file: {all_output_file}")
            else:
                try:
                    os.remove(all_output_file)
                except:
                    pass
    
    print(f"\n{'='*60}")
    print(f"All files created in: {save_dir}")
    print(f"{'='*60}")
    root.destroy()
    return save_dir

if __name__ == "__main__":
    print("Starting Excel file merge process...")
    
    # Group by concentration (creates separate files for each gelatin/particle combo)
    save_dir = merge_excel_files(merge_metrics=True, group_by_concentration=True)
    
    # To merge everything into one file regardless of concentration:
    # save_dir = merge_excel_files(merge_metrics=True, group_by_concentration=False)
    
    if save_dir:
        print(f"\n✓ Process completed successfully!")
        print(f"  Files saved to: {save_dir}")
    else:
        print("Process was cancelled or encountered an error.")

Starting Excel file merge process...
  Path: C:/Users/Talaial Alina/OneDrive - UCB-O365/Courses/Year 1/Fall Semester Aug-Dec 2020/CHEN 5840 - Independent Study/Hydrogels/Gel microscopy/06JUL25/8% gelatin\DBPC HMSN\0 cycles\stitched_image\metrics\intensity_range_summary.xlsx
  Extracted metadata: {'particle_type': 'DBPC HMSN', 'cycles': '0', 'gelatin_conc': '8'}
  Path: C:/Users/Talaial Alina/OneDrive - UCB-O365/Courses/Year 1/Fall Semester Aug-Dec 2020/CHEN 5840 - Independent Study/Hydrogels/Gel microscopy/06JUL25/8% gelatin\DBPC HMSN\11 cycles\stitched_image\metrics\intensity_range_summary.xlsx
  Extracted metadata: {'particle_type': 'DBPC HMSN', 'cycles': '11', 'gelatin_conc': '8'}
  Path: C:/Users/Talaial Alina/OneDrive - UCB-O365/Courses/Year 1/Fall Semester Aug-Dec 2020/CHEN 5840 - Independent Study/Hydrogels/Gel microscopy/06JUL25/8% gelatin\DBPC HMSN\28 cycles\stitched_image\metrics\intensity_range_summary.xlsx
  Extracted metadata: {'particle_type': 'DBPC HMSN', 'cycles': '28',